In [1]:
%matplotlib widget

In [1]:
from pathlib import Path
import numpy as np
import flammkuchen as fl
import pandas as pd

import skimage as sk
import json

from split_dataset import SplitDataset
from fimpylab.core.lightsheet_experiment import LightsheetExperiment

from matplotlib import  pyplot as plt
import seaborn as sns
from tqdm import tqdm
sns.set(style="ticks", palette="deep")
cols = sns.color_palette()
import ipywidgets as widgets

from lotr.pca import pca_and_phase, fictive_heading_and_fit, fit_phase_neurons
from circle_fit import hyper_fit
from lotr import LotrExperiment

import lotr.plotting as pltltr
COLS = pltltr.COLS

from lotr.utils import roll_columns_jit

ModuleNotFoundError: No module named 'lotr'

In [ ]:
def nan_phase_jumps(phase_array):
    out_array = phase_array.copy()
    out_array[1:][np.abs(np.diff(out_array)) > np.pi] = np.nan
    return out_array

In [ ]:
master = Path(r"Z:\Hagar\E0040\2p ablations\2p ablations near ipn")

fish_list = list(master.glob("*_f*"))
path = fish_list[1]

traces = fl.load(path / "filtered_traces.h5", "/detr")
traces_full = fl.load(path / "filtered_traces.h5", "/detr")
selected = fl.load(path / "selected.h5")
traces_hdn = traces_full[:, selected]

suite2p_data = fl.load(path / "data_from_suite2p_unfiltered.h5")
roi_map = suite2p_data['rois_stack']
coords = suite2p_data['coords']
anatomy = suite2p_data['anatomy_stack']

df = fl.load(path / "bouts_df.h5")
exp = LotrExperiment(path)
fs = int(exp.fn)
beh_df = exp.behavior_log

t_start_s = 50
t_lims = (t_start_s*exp.fn, exp.n_pts)
t_slice = slice(*t_lims)

In [ ]:
pca_scores, angles, _, _ = pca_and_phase(traces[t_slice, selected].T, traces[t_slice, selected].T)
pcaed, phase, _, _ = pca_and_phase(traces[t_slice, selected], traces[:, selected])

In [ ]:
unwrapped_phase = np.unwrap(phase)
traj, params = fictive_heading_and_fit(unwrapped_phase, df, min_bias=0.1) 

exp = LotrExperiment(path)
coords = exp.coords_um[selected, 1:]

sort_idxs = np.argsort(exp.rpc_angles)
phase_shifts = (exp.network_phase / (2 * np.pi)) * (exp.n_hdns - 1)

In [ ]:
t = np.arange(np.shape(traces)[1]) / fs
len_rec, num_traces = np.shape(traces)
print("num_traces: ", num_traces)
print("len_rec: ", len_rec)
print("sampling rate: ", fs)

In [ ]:
# figure 1 - HDN neurons
xlim1 = 0
xlim2 = len_rec // exp.fs //2
fig_hdn, ax_hdn = plt.subplots(2, 2, figsize=(8, 3), gridspec_kw={'width_ratios': [1, 2], 'height_ratios': [1, 6]}) 

############ Anatomy plot of HDNs:
for i in range(2):
    for j in range(2):
        ax_hdn[i,j].axis('off')

anatomy_layer = np.mean(anatomy, axis=0)
anatomy_layer = np.rot90(anatomy_layer, k=2, axes=(1, 0))
colored_rois = exp.color_rois_by(np.ones(len(selected))*5, indexes=selected, color_scheme='rainbow')
colored_rois = exp.color_rois_by(angles, indexes=selected, color_scheme=COLS["phase"])
ax_hdn[1, 0].imshow(anatomy_layer, extent=exp.plane_ext_um, origin="upper", cmap='gray_r')

ax_hdn[1, 0].imshow(np.rot90(colored_rois.max(0), 3), extent=exp.plane_ext_um, origin="lower", alpha=1)
pltltr.add_anatomy_scalebar(ax_hdn[1, 0], pos=(-10,-10))

############ Tail:
t_beh = np.asarray(beh_df["t"])
t_beh[-1]
ax_hdn[0, 1].plot(t_beh, beh_df["tail_sum"], color=cols[7], label='Tail', rasterized=True)
ax_hdn[0, 1].legend(loc=2, bbox_to_anchor=(0.8, 2), fontsize=7)
ax_hdn[0, 1].set_aspect('auto')
ax_hdn[0, 1].set_xlim(xlim1, xlim2)

############ Sorted traces:
N_BINS_RESAMPLED = 100
resampling_base = np.linspace(-np.pi, np.pi, N_BINS_RESAMPLED)
angle_resampled_traces = np.zeros((exp.n_pts, N_BINS_RESAMPLED))
for i in range(exp.n_pts):
    angle_resampled_traces[i, :] = np.interp(
        resampling_base,
        exp.rpc_angles[sort_idxs],
        exp.traces[i, exp.hdn_indexes[sort_idxs]],
    )

phase_shifts_resamp = (exp.network_phase / (2 * np.pi)) * (N_BINS_RESAMPLED - 1)
shifted_traces_resamp = roll_columns_jit(
    angle_resampled_traces, -np.round(phase_shifts_resamp)
)

ax = ax_hdn[1, 1]
im = ax.imshow(
    angle_resampled_traces.T,
    extent=[0, exp.time_arr[-1], -np.pi, np.pi],
    aspect="auto",
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=2.0,
)

pltltr.add_dff_cbar(
    im,
    ax,
    (1.07, 0.04, 0.03, 0.5),
    title="ΔF (Z.)",
    titlesize=6,
    labelsize=5,
    ticklabels=None,
)

pltltr.despine(ax, ["left", "right", "top", "bottom"])
ax.set(ylabel="ROI angle", **pltltr.get_pi_labels(0.5, ax="y"))

line = nan_phase_jumps(phase_shifts) + exp.n_hdns / 3
line = line - np.nanmin(line)
line /=(np.nanmax(line))
line *= np.pi * 2
line -= np.pi

t_line = np.arange(0, np.shape(line)[0]) / 3
ax.plot(t_line, line, lw=1, c=cols[2], label="Network phase")
ax.legend(loc=2, bbox_to_anchor=(0.65, 1.15), fontsize=7)
ax.set_xlim(xlim1, xlim2)


In [ ]:
file_name = path / "HDN sorted phase color.jpg"
fig_hdn.savefig(file_name, dpi=300)
file_name = path / "HDN sorted phase color.pdf"
fig_hdn.savefig(file_name, dpi=300)

In [ ]:
########## HDN and estimated heading 

In [ ]:
pcaed, phase, _, _ = pca_and_phase(traces[t_slice, selected], traces[:, selected])
unwrapped_phase = np.unwrap(phase)

traj, params = fictive_heading_and_fit(unwrapped_phase, df, fn=exp.fn, min_bias=0.1)

r_value = np.corrcoef(traj[:], unwrapped_phase[:])[1,0]
r_value = np.round(r_value, decimals=2)

In [ ]:
r_values = [-0.8, -0.95, -0.89]

print(np.mean(r_values))
print(np.std(r_values)/3)